# Section 3: Experiments with Graph-based Learning

In this section, we will build several Graph-based MLs. There are several types of ML with Graphs -- Feature Engineering (for Graph Characteristics as features), Graph Representation Learning, Graph Neural Networks, and more. Here, we will implement and compare some of them.

    3a. A Graph Convolutional Network (GCN)
    3b. A Graph Attention Network (GAT)


**The Dataset** we will use is the CiteSeer Dataset and classify the documents or the nodes. This dataset is a popular benchmark for Graph-based MLs. As of January 2025, the best accuracy achieved is **82.07 ± 1.04** by ["ACMII-Snowball-2"](https://paperswithcode.com/paper/is-heterophily-a-real-nightmare-for-graph). A live update on the rankings can be found in this [link](https://paperswithcode.com/sota/node-classification-on-citeseer).

Can we beat it? Perhaps not so easily, as brilliant ML scientists and engineers have already thrown the kitchen sink at it. But we can definitely try! Why not dream? We will see how close we can get.

The information within the dataset: This dataset contains a set of 3327 scientific papers represented by binary vectors of 3703 words, with the values represent the presence or absence of the words in the document. A **key feature** of the dataset is that it also contains data on the citations among the papers as a citation graph or network, along with the text data. Here we are only use the text data. In later sections, we will incorporate the Graph data and see how it changes things. The availability of both types of data is the biggest reason we picked this dataset.

<!-- **The General Plan**: -->
<!-- 1. <u>Build a Modeling Pipeline</u>: For each model, we will create a "pipeline". The pipelines can include everything between inputs and outputs. For example, we may want to represent our texts as certain kind of vectors (e.g., one_hot, TF-IDF). Then, We may want to transform our vectors and reduce their dimensions using methods such as Singular Value Decomposition (SVD) or Non-negative Matrix Factorization (NMF). Finally, we would have our model to feed these all into. This workflow can be conveniently represented as a pipeline, as we will see.

2. <u>Train, Validate, and Test</u>: After training, we will check the validation and the test accuracies. 

3. <u>Save the Models</u>: We will then save the models so that we can call them up again in later sections.

It is almost as simple as it sounds. Of course, there are some nuances to these methods. But, we do not need to worry too much about it now. We will discuss things as they become necessary.

Enough talking! Let's get started! -->

## A Brief Intro to Machine Learning with Graphs
Learning from graphs is an age-old approach (tracing back to Leonhard Euler!) that has seen a (relatively) recent resurgence for applications in Deep Learning. We briefly discuss the different types and how they relate to traditional Machine Learning. For details, a great open-source resource for ML with Graphs is the CS224W course by Jure Lescovec (https://web.stanford.edu/class/cs224w/). The lecture slides, notebooks, and problem sets for several years are provided in entirety, such as the Fall 2023 resource can be found here: https://snap.stanford.edu/class/cs224w-2023/. Lecture videos for particular years can be found on YouTube. Here is the lecture series for 2021: https://youtu.be/JAB_plj2rbA?si=ujCiFl6HBFSqlGVf

**What is a Graph?**

A graph or a network can be considered as set of variables as **nodes** or **vertices** that are connected by some relationships represented by **edges**. Graphs are generally denoted by $G = (V, E, W)$ where V is a set of vertices or nodes, E is the set of edges connecting them, and W represent the edge weights or strengths of relationship between nodes.

The graphs are **undirected** if the edges have no direction or **directed** if they do. If the edge weights are binary (1 if a connection exists, 0 if not), the graphs are called **unweighted** -- somewhat paradoxically, I must admit -- as the weights simply mean the existence of connections and with no information on how strong are the relationships. When the weights are non-binary and represent, the graphs are **weighted graphs**.

The graph in the CiteSeer Dataset is an **undirected, unweighted** graph, in essence, the simplest kind. As we will see, the incorporation of the edge information after setting each document as a node, will help us exceed the accuracies we had achieved with the shallow and the deep networks.

### Different types of Learning from Graphs

**Feature Engineering to encode information from graphs**

In this case, we develop new features that capture the information contained within the graphs and use it in a traditional machine learning pipeline.

    * Manual Feature Extraction: Extracting features like node degree, clustering coefficient, and centrality measures.
    * Graph Kernels: Methods like Weisfeiler-Lehman Kernel and Shortest Path Kernel that measure similarity between graphs.
    * Graph Augmentation: Techniques that enhance graph data by adding or modifying nodes and edges.

**Graph Representation Learning**

    1. Node Embeddings: Techniques like DeepWalk or node2vec that learn low-dimensional representations of nodes.
    2. Edge Embeddings: Techniques that focus on learning representations for edges in a graph, perhaps for a link or edge prediction task.
    3. Graph Embeddings: Methods like GraphSAGE and Graph Convolutional Networks (GCNs) that learn representations for entire graphs.

The representations can also be learned from the embedding layer of the neural networks trained for a downstream task. For example, the node embeddings can be learned from an RNN or a GCN trained to predict the labels. Therefore, in essence, any of the following approaches can be used for graph representation learning. Moreover, the learned representation themselves can be reused for other downstream tasks, such as community detection, clustering, and even to train a supervised/semi-supervised classification model using the embeddings from the already trained model. We will explore this approach in one future experiment.

**Graph Neural Networks (GNNs)**
    
    * Graph Convolutional Networks (GCNs): Neural networks that apply convolution operations on graphs.
    * Graph Attention Networks (GATs): GNNs that use attention mechanisms to weigh the importance of neighboring nodes.
    * Graph Recurrent Networks (GRNs): GNNs that incorporate recurrent neural network architectures for sequential data.
    * Graph Autoencoders: Unsupervised learning models that encode graph data into a latent space and then decode it back.

**Graph-Based Algorithms**:
    
    * PageRank: An algorithm used to rank nodes in a graph based on their importance.
    * Label Propagation: A semi-supervised learning algorithm that propagates labels through the graph.
    * Community Detection: Algorithms like Louvain and Girvan-Newman that identify clusters or communities within a graph.

**Graph Generative Models**
    
    * GraphRNN: A recurrent neural network-based model for generating graphs.
    * GraphVAE: A variational autoencoder for generating graphs.
    * GraphGAN: A generative adversarial network for generating graphs.

**Some Applications**

    * Social Network Analysis: Analyzing social networks to understand relationships and influence.
    * Recommendation Systems: Using graph-based techniques to recommend items or connections.
    * Molecular Graphs: Analyzing molecular structures for drug discovery and chemistry.
    * Knowledge Graphs: Representing and reasoning over knowledge in a structured form.

These are just a few examples of the diverse techniques and applications in graph-based ML. We can find more details in the [Hugging Face blog](https://huggingface.co/blog/intro-graphml).

In [1]:
# First thing, get some essential Packages
# We also create a new directory to save the models

# Numpy for matrices
import numpy as np
import pandas as pd
np.random.seed(0)

# Visualization
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import itertools
from collections import Counter

import os

# Define the name of the directory to be created
directory_name = "Saved_ML_models_Exp1"

# Get the current working directory
current_working_directory = os.getcwd()
# Create the full path for the new directory
new_directory_path = os.path.join(current_working_directory, directory_name)

# Check if the directory exists, and create it if it does not
if not os.path.exists(new_directory_path):
    os.makedirs(new_directory_path)
    print(f"Directory '{directory_name}' created at {new_directory_path}")
else:
    print(f"Directory '{directory_name}' already exists at {new_directory_path}")


Directory 'Saved_ML_models_Exp1' already exists at c:\Users\rouss\Documents\GitHub\Many_MLs_for_Node_Classification\Saved_ML_models_Exp1


## Get the CiteSeer Dataset
This dataset is available through PyTorch Geometric, a package dedicated to Graph NNs. The CiteSeer is one of the several datasets available.

In [2]:
from torch_geometric.datasets import Planetoid

# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="CiteSeer")

data = dataset[0] # We extract the data we need.

In [3]:
# Print information about the dataset
print("Dataset name:", dataset)
print("Input Text Data shape:", data.x.shape)
print("First five rows of the text data:\n", data.x[0:5, :])

Dataset name: CiteSeer()
Input Text Data shape: torch.Size([3327, 3703])
First five rows of the text data:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


As for the **text data**, the dataset has 3327 documents as rows, made up of 3703 unique words. The documents are represented as one-hot vectors of length 3703. One hot vectors simply mean that if a word exists, then we assign it's magnitude to be 1 and if not, then we assign the magnitude to be 0. We just to need to follow the same order of words for each document, and that is it.

What about the **graph information** provided in the Dataset? The edge information can be accessed as show below and is presented in Coordinate list format. Basically, we are given the coordinates of the nodes connected to each other, separated in two lists of nodes (e.g., 628 to 0, 158 to 1, 486 to 1 and so on). As we see, there are 9104 edges between the 3327 documents we have in the dataset.

There are different ways to represent the same information, such as using Adjacency matrices or Adjacency lists. To me, the adjacency matrix is the easiest to internalize (see example below), in which the row and the column indices are the coordinates of nodes and the matrix contains just the weights for each pair of nodes.

Now, we know a lot about the data we have. Time to just build the GNNs!

In [24]:
print("Edges as coordinate lists:")
print(data.edge_index)
print(data.edge_index.shape)

# Create Adjacency matrix
coo_list = data.edge_index

adj_matrix = np.zeros([data.x.shape[0], data.x.shape[0]])
for (i,j) in zip(coo_list[0,:], coo_list[1,:]): adj_matrix[i, j] = 1

print("\nAdjacency Matrix:\n", adj_matrix)

Edges as coordinate lists:
tensor([[ 628,  158,  486,  ..., 2820, 1643,   33],
        [   0,    1,    1,  ..., 3324, 3325, 3326]])
torch.Size([2, 9104])

Adjacency Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Load Necessary Packages

In [26]:
# We assume that PyTorch is already installed
import torch
torchversion = torch.__version__

# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv

from torch_geometric.utils import remove_isolated_nodes
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from torch_geometric.utils import degree

## Model 3a: Building a Graph Convolutional Network

First, we need to build the network. The below class implements it and is written in pretty much standard ways of building a PyTorch model. The first function in the class defines the layers we want in the model and the number of inputs and outputs for each layer. The second one defines the transformations and passing of the information from one layer to another, thus completing the network architecture.

The beauty of our graph-based ML model lies in the "GCNConv" layers from PyTorch. Similar to convolutions in CNNs, GCNs combine information for a node from its neighbors in attempt to find helpful, complex relationships within the data. To do sc, it combines the node features (i.e., the words in our documents) with the information of the edges between nodes. The rest of the layers and also the architecture are similar to the standard Deep Learning counterparts.

A last point to note is the input variables to each function. For the init function, we need to provide the input data dimension (i.e., the number of words or features), the number of nodes in the hidden layer (our choice!), and the output dimension (i.e., the number of classes to predict). For the forward function, we provide both the text data (in "x") and the edge info (in "edge_index") to our model. The text data is presented in one-hot encoded form and the edge info is presented as a co-ordinate list, just as we have in our dataset.

In [27]:
class GCN(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.gcn1 = GCNConv(dim_in, dim_h)
    self.gcn2 = GCNConv(dim_h, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.01,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.5, training=self.training) # Note: We are using dropouts
    h = self.gcn1(h, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.gcn2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

Next, we set up our (1) accuracy, (2) train, and (3) test/evaluation functions

In [51]:
def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 200

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        _, out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(out[data.train_mask].argmax(dim = 1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim = 1), data.y[data.val_mask])

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                  f'Val Acc: {val_acc*100:.2f}%')

    return(model)

torch.no_grad() # Because we are testing, we do not need gradient calculation
def evaluation(model, data, data_mask_list = ["train_mask", "val_mask", "test_mask"]):
    model.eval()

    # We get the predictions out for ALL DOCUMENTS first!
    _, out_pred_labels = model(data.x, data.edge_index)

    # Evaluate by train, val, and test sets

    list_of_acc_train_val_test = [] # to save the accuracies for different sets
    for mask in data_mask_list:
        acc = accuracy(out_pred_labels[data[mask]].argmax(dim = 1), data.y[data[mask]])
        acc = acc*100 # convert to percentage
        list_of_acc_train_val_test.append(acc)
    
    return(list_of_acc_train_val_test)

### Train, Vaildate, and Test model

In [57]:
%%time

# 1. Create an untrained GCN model
gcn_model_3a = GCN(dataset.num_features, 32, dataset.num_classes)
print(gcn_model_3a)

# 2. Train or Fit the model to data
train(gcn_model_3a, data)

# 3. Check Accuracies on TRAIN-VALIDATION-TEST sets
list_acc = evaluation(gcn_model_3a, data)
print("\n" + "--"*10)
print('GCN Train-Val-Test Accuracy (in % accuracy): \n', list_acc)

GCN(
  (gcn1): GCNConv(3703, 32)
  (gcn2): GCNConv(32, 6)
)
Epoch   0 | Train Loss: 1.812 | Train Acc:  12.50% | Val Loss: 1.79 | Val Acc: 17.60%
Epoch  10 | Train Loss: 0.235 | Train Acc:  95.00% | Val Loss: 1.17 | Val Acc: 61.20%
Epoch  20 | Train Loss: 0.069 | Train Acc:  99.17% | Val Loss: 1.40 | Val Acc: 59.20%
Epoch  30 | Train Loss: 0.030 | Train Acc:  98.33% | Val Loss: 1.48 | Val Acc: 61.40%
Epoch  40 | Train Loss: 0.018 | Train Acc: 100.00% | Val Loss: 1.58 | Val Acc: 59.60%
Epoch  50 | Train Loss: 0.023 | Train Acc: 100.00% | Val Loss: 1.54 | Val Acc: 59.40%
Epoch  60 | Train Loss: 0.028 | Train Acc:  99.17% | Val Loss: 1.51 | Val Acc: 59.00%
Epoch  70 | Train Loss: 0.025 | Train Acc:  99.17% | Val Loss: 1.34 | Val Acc: 62.20%
Epoch  80 | Train Loss: 0.033 | Train Acc:  99.17% | Val Loss: 1.46 | Val Acc: 59.40%
Epoch  90 | Train Loss: 0.017 | Train Acc: 100.00% | Val Loss: 1.44 | Val Acc: 62.60%
Epoch 100 | Train Loss: 0.024 | Train Acc: 100.00% | Val Loss: 1.44 | Val Acc: 6

## Model 3b: Building a Graph Attention Network

The below class implements the GAT. Once again, the first function in the class defines the layers in the model and the number of inputs and outputs for each layer. The second one defines the transformations and message-passing from one layer to another, completing the network.

Similar to "GCNConv", PyTorch provides "GATv2Conv" layers. As the name suggests, GAT uses the powers of convolution but integrates with an "attention" mechanism. For this reason, whereas GCNs are closely related to CNNs, GATs are similar to Transformers and its attention network. In fact, Transformers can even be imagined as [a special case of GNNs](https://mlabonne.github.io/blog/posts/2022-03-09-Graph_Attention_Network.html). The GATs use an attention mechanism with the observation that not all nodes are equally important and the amounts of attention to each node can be varied based on their importance to improve computational efficiency and prediction accuracy. We do it by creating "attention heads" in charge of attending to the nodes based on their importance. Please see [this tutorial by Maxime Labonne](https://mlabonne.github.io/blog/posts/2022-03-09-Graph_Attention_Network.html) for a visualization of the process and some details about the mathematical transformation involved in the process.

To note, this notion of varying importance of nodes is present in many areas of Graph Theory and Graph-based MLs. For example, we could have calculated the node importance as PageRank values and used it as a feature in a traditional machine learning pipeline. However, it is a tricky concept due to the different ways a node could be important, so the choice of importance measures is sophisticated and depends on applications. Fortunately, Graph representation learning and the GNNs both learn the representations directly from data, thus avoiding the difficulties associated with a feature engineering approach.

There are many more interesting points to think about in the complex but beautiful world of Graphs. However, here we will focus on implementation first to build a solid foundation to convert our thoughts about relationships in graphs to actions.

Again, a last point to note before we start modeling is the input variables to each function. For the init function, we do the same as before -- provide the input data dimension (i.e., the number of words or features), the number of nodes in the hidden layer (our choice!), and the output dimension (i.e., the number of classes to predict) -- but also need to provide the number of attention heads. Higher number of heads will increase computational load and computation time. The way we set up the codes, its value can be set while specifying the models. Here, we experiment between 8 or 16 heads. For the forward function, we do the same as we did with GCNs -- the text data is passed to "x" and the edge info to "edge_index".

<!-- A last point to note is the input variables in the forward function. We provide both the text data (in "x") and the edge info (in "edge_index") to our model. The text data is presented in one-hot encoded form and the edge info is presented as a co-ordinate list, just as we have in our dataset. -->

In [63]:
class GAT(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out, heads=16):
    super().__init__()
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.005,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.5, training=self.training)
    h = self.gat1(x, edge_index)
    h = F.elu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.gat2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

We already have our accuracy, train, and evaluation functions set up and we can go directly to training our model.

In [62]:
%%time

# 1. Create an untrained gat model
gat_model_3b = GAT(dataset.num_features, 32, dataset.num_classes, heads = 8)
print(gat_model_3b)

# 2. Train or Fit the model to data
train(gat_model_3b, data)

# 3. Check Accuracies on TRAIN-VALIDATION-TEST sets
list_acc = evaluation(gat_model_3b, data)
print("\n" + "--"*10)
print('GAT Train-Val-Test Accuracy (in % accuracy): \n', list_acc)

GAT(
  (gat1): GATv2Conv(3703, 32, heads=8)
  (gat2): GATv2Conv(256, 6, heads=1)
)
Epoch   0 | Train Loss: 1.791 | Train Acc:  23.33% | Val Loss: 1.77 | Val Acc: 25.00%
Epoch  10 | Train Loss: 0.004 | Train Acc: 100.00% | Val Loss: 1.32 | Val Acc: 63.40%
Epoch  20 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 1.72 | Val Acc: 59.80%
Epoch  30 | Train Loss: 0.000 | Train Acc: 100.00% | Val Loss: 1.63 | Val Acc: 62.40%
Epoch  40 | Train Loss: 0.002 | Train Acc: 100.00% | Val Loss: 1.46 | Val Acc: 63.40%
Epoch  50 | Train Loss: 0.003 | Train Acc: 100.00% | Val Loss: 1.31 | Val Acc: 63.80%
Epoch  60 | Train Loss: 0.004 | Train Acc: 100.00% | Val Loss: 1.19 | Val Acc: 67.20%
Epoch  70 | Train Loss: 0.005 | Train Acc: 100.00% | Val Loss: 1.13 | Val Acc: 67.00%
Epoch  80 | Train Loss: 0.005 | Train Acc: 100.00% | Val Loss: 1.14 | Val Acc: 68.00%
Epoch  90 | Train Loss: 0.005 | Train Acc: 100.00% | Val Loss: 1.17 | Val Acc: 68.20%
Epoch 100 | Train Loss: 0.004 | Train Acc: 100.00% | Val 

As we see, the GATs are much slower than GCNs. In our case, the GATs did not improve the accuracy by much over GCNs either. However, GATs have been found to consistently outperform the accuracies of GCNs. 

## What comes next?
We will try to "hide" information from our GCNs and GATs, and see how well do they do when presented with only text or only edge information. Exciting, right?